# Problem Set 2
## Cecilia Barnes

### 1. Legality
1. https://en.wikibooks.org/wiki/Cookbook:Recipes is the website I want to scrape

2. In wikibooks terms of service, it does mention data collections in their terms of use. It says their " mission is to empower and engage people around the world to collect and develop content under a free license or in the public domain, and to disseminate it effectively and globally, free of charge." It mentions that users are free to read and print articles, share and reuse articles under free licenses, and contribute to and edit their sites.

3. Steps to lessen the burden of the provider are to only request the page info once unless you need to entirely rerun the code. I could donate money to wikimedia as well since it is a nonprofit site.


### 2. Scrape Recipes

In [2]:
from bs4 import BeautifulSoup
import requests
link = 'https://en.wikibooks.org/wiki/Cookbook:Recipes'
result = requests.get(link)

In [3]:
#wiki = BeautifulSoup(result.content)
wiki = BeautifulSoup(result.text)

In [4]:
li = wiki.select('li')
links = [a.find('a') for a in li]
hrefs = []
for each in links[:50]:
    hrefs.append(each.get('href'))
    #print(each['href'])

In [5]:
import re
import pandas as pd
base_url = "https://en.wikibooks.org/"
# make dataframe w recipe id, name, text, quantity, unit, ingredient
df_dict = {"id":[], "name":[], "text":[], "quantity":[], "unit":[], "ingredient":[]}
recipe_id = 0

for each in links[:50]:
    recipe_pg = requests.get(base_url + each['href'])
    soup = BeautifulSoup(recipe_pg.text)
    recipe_id = recipe_id
    name = soup.h1.text.split(":")[1]
    # get list of ingredients
    #check that item above is h3 with id=ingredients
    ul = soup.find_all("ul")
    ingredient_ul = []
    for each in ul:
        h2 = each.find_previous_sibling('h2')
        h3 = each.find_previous_sibling('h3')
        if h2 is not None:
            if "Ingredient" in h2.text:
                ingredient_ul.append(each)
        if h3 is not None:
            if "Ingredient" in h3.text:
                ingredient_ul.append(each)
    ingredients = []
    for ul in ingredient_ul:
        ingredients.append(ul.find_all("li"))
    #ingredients = ingredient_ul.find_all("li")
    units = ["cups", "cup", "tablespoons", "tablespoon", "Tablespoon", "teaspoons", "teaspoon", "tsp", "Tbs", "oz", "tbsp"]
    # for each ingredient, make list and add to dict
    for item in ingredients:
        for li in item:
            # getting full line of text about ingredient
            text = li.text
        
            # finding digits in text
            quantity = re.findall("^\d*½*/*-*–*\d*", text)
            if len(quantity) > 0:
                quantity = quantity[0]
            else:
                quantity = ""

            # finding units in text
            unit_found = re.findall('|'.join(units), text)
            if len(unit_found) > 0:
                unit = re.findall('|'.join(units), text)[0]
            else:
                unit = ""

            # finding ingredients in text
            if quantity == "" and unit == "":
                ingredient = text
            elif quantity != "" and unit == "":
                ingredient = text.split(quantity)[1]
            elif quantity == "" and unit != "":
                ingredient = text.split(unit)[1]
            else:
                ingredient = text.split(unit)[1]
            if "of" in ingredient:
                ingredient = ingredient.replace("of ", "")
                
        # add all features to dict
        df_dict["id"].append(recipe_id)
        df_dict["name"].append(name)
        df_dict["text"].append(text)
        df_dict["quantity"].append(quantity)
        df_dict["unit"].append(unit)
        df_dict["ingredient"].append(ingredient)
    recipe_id += 1
#df_dict
df = pd.DataFrame(df_dict)

In [6]:
df

,id,name,text,quantity,unit,ingredient
0,0,1-2-3-4 Cake,4 eggs,4,,eggs
1,1,Green Mango and Cumin Drink (Aam Panna),A handful of fresh mint leaves,,,A handful fresh mint leaves
2,2,Acorn Squash with Orange Marmalade,1/8 teaspoon salt,1/8,teaspoon,salt
3,3,Shrimp Fritters (Accra),black pepper and salt (really optional here) t...,,,black pepper and salt (really optional here) t...
4,4,Adobo,½ tsp (2.5 mL) freshly cracked (not ground) bl...,½,tsp,(2.5 mL) freshly cracked (not ground) black p...
5,5,Agedashi Tofu,Oil for deep frying,,,Oil for deep frying
6,5,Agedashi Tofu,1½ Tbs (22.5 mL) mirin,1½,Tbs,(22.5 mL) mirin
7,5,Agedashi Tofu,Grated daikon (white radish),,,Grated daikon (white radish)
8,6,Alfredo Sauce,1 pound (450 g) fettuccine egg noodles,1,,pound (450 g) fettuccine egg noodles
9,6,Alfredo Sauce,pinch of fresh-ground nutmeg,,,pinch fresh-ground nutmeg


In [7]:
df.to_csv("recipe_info.csv")

One of the issues I found that I couldn't easily solve was fully isolating the ingredient itself. Many of my ingredients still have extraneous text attached to them (like the measurement in grams). It is nearly impossible to isolate only the ingredient itself.

### 3. Sweet or Salty? 

In [8]:
sweet = df[df["ingredient"] == "sugar"]
salty = df[df["ingredient"] == "salt"]
if len(sweet) >= len(salty):
    print("1. There are more (or equal) sweet recipes than salty ones.")
else:
    print("1. There are less sweet recipes than salty ones.")

1. There are more (or equal) sweet recipes than salty ones.


2. There are problems with this since the units in each recipe are so varied. You would have to choose one unit and convert all recipes to that unit and then compare, which isn't possible given the weird quanities like "a handful". Additionally, this doesn't take into account the amount of sugar in each of the ingredients. For example, chocolate might have 25 grams of sugar that cannot be accounted for because we do not have the nutrition facts of each ingredient and therefore absolute values cannot be accurately compared.


### 4. Favorite Recipe
My favorite recipe is chocolate chip cookies but I am out of groceries right now so I cant make them :(